Notes:
 * SOTA (run 14) on the sample TICs

In [1]:
import os

chkpt_root = '/mnt/tess/astronet/checkpoints/extended_23_run_14'
data_files = '/mnt/tess/astronet/tfrecords-s33-cam1-sample2/*'
tces_file = '/mnt/tess/astronet/tces-s33_cam1_sample.csv'

nruns = 10

def load_ensemble(chkpt_root, nruns):
    checkpts = []
    for i in range(nruns):
        parent = os.path.join(chkpt_root, str(i + 1))
        if not os.path.exists(parent):
            break
        all_dirs = os.listdir(parent)
        if not all_dirs:
            break
        d, = all_dirs
        checkpts.append(os.path.join(parent, d))
    return checkpts

paths = load_ensemble(chkpt_root, nruns)
paths

['/mnt/tess/astronet/checkpoints/extended_23_run_14/1/AstroCNNModel_extended_20210131_212427',
 '/mnt/tess/astronet/checkpoints/extended_23_run_14/2/AstroCNNModel_extended_20210131_215717',
 '/mnt/tess/astronet/checkpoints/extended_23_run_14/3/AstroCNNModel_extended_20210131_222957',
 '/mnt/tess/astronet/checkpoints/extended_23_run_14/4/AstroCNNModel_extended_20210131_230255',
 '/mnt/tess/astronet/checkpoints/extended_23_run_14/5/AstroCNNModel_extended_20210131_233546',
 '/mnt/tess/astronet/checkpoints/extended_23_run_14/6/AstroCNNModel_extended_20210201_000826',
 '/mnt/tess/astronet/checkpoints/extended_23_run_14/7/AstroCNNModel_extended_20210201_004107',
 '/mnt/tess/astronet/checkpoints/extended_23_run_14/8/AstroCNNModel_extended_20210201_011350',
 '/mnt/tess/astronet/checkpoints/extended_23_run_14/9/AstroCNNModel_extended_20210201_014655',
 '/mnt/tess/astronet/checkpoints/extended_23_run_14/10/AstroCNNModel_extended_20210201_021945']

In [2]:
import getpass
import os
from astronet import predict
import tensorflow as tf


def run_predictions(path):
    predict.FLAGS = predict.parser.parse_args([
      '--model_dir', path,
      '--data_files', data_files,
      '--output_file', '',
    ])

    return predict.predict()


paths = load_ensemble(chkpt_root, nruns)
ensemble_preds = []
config = None
for i, path in enumerate(paths):
    print(f'Running model {i + 1}')
    preds, config = run_predictions(path)
    ensemble_preds.append(preds.set_index('tic_id'))
    print()

Running model 1
Binary prediction threshold: 0.2152499407880693 (orientative)
757 records
Running model 2
Binary prediction threshold: 0.2152499407880693 (orientative)
757 records
Running model 3
Binary prediction threshold: 0.2152499407880693 (orientative)
757 records
Running model 4
Binary prediction threshold: 0.2152499407880693 (orientative)
757 records
Running model 5
Binary prediction threshold: 0.2152499407880693 (orientative)
757 records
Running model 6
Binary prediction threshold: 0.2152499407880693 (orientative)
757 records
Running model 7
Binary prediction threshold: 0.2152499407880693 (orientative)
757 records
Running model 8
Binary prediction threshold: 0.2152499407880693 (orientative)
757 records
Running model 9
Binary prediction threshold: 0.2152499407880693 (orientative)
757 records
Running model 10
Binary prediction threshold: 0.2152499407880693 (orientative)
757 records


In [31]:
labels = ['disp_E', 'disp_N', 'disp_J', 'disp_S', 'disp_B']

col_e = labels.index('disp_E')
# thresh = config.hparams.prediction_threshold
# thresh = 0.030485098838860747  # From the validation numbers - maximum thrershold for 100% recall
thresh = 0.31245827674871207  # Relaxed to match Liang's precision value

In [32]:
import numpy as np
import pandas as pd

agg_preds = {}

for preds in ensemble_preds:
    for tic_id in preds.index:
        if tic_id not in agg_preds:
            agg_preds[tic_id] = []

        row = preds[preds.index == tic_id]
        pred_v = row.values[0]
        if len(row.values) > 1:
            print(f'Warning: duplicate predictions for {tic_id}')
        if pred_v[col_e] >= thresh:
            agg_preds[tic_id].append('disp_E')
        else:
            masked_v = [v if i != col_e else 0 for i, v in enumerate(pred_v)]
            agg_preds[tic_id].append(preds.columns[np.argmax(masked_v)])

In [33]:
final_preds = []
for tic_id in list(agg_preds.keys()):
    counts = {l: 0 for l in labels}
    for e in agg_preds[tic_id]:
        counts[e] += 1
    maxcount = max(counts.values())
    counts.update({
        'tic_id': tic_id,
        'maxcount': maxcount,
    })
    final_preds.append(counts)

final_preds = pd.DataFrame(final_preds).set_index('tic_id')

In [34]:
final_preds.head()

,disp_E,disp_N,disp_J,disp_S,disp_B,maxcount
tic_id,,,,,,
11690157,0,0,10,0,0,10
232479493,0,0,10,0,0,10
220338896,0,0,10,0,0,10
265446651,0,0,10,0,0,10
251192557,10,0,0,0,0,10


In [35]:
def compare(ensemble_preds, filter):
    result = ensemble_preds[0][filter]
    for preds in ensemble_preds[1:]:
        result = result.append(preds[filter])
    return result

compare(ensemble_preds, preds.index == 263337671)

,disp_E,disp_N,disp_J,disp_S,disp_B
tic_id,,,,,
263337671,0.002818,0.020683,0.904438,0.263909,0.003210
263337671,0.014255,0.030863,0.666594,0.534820,0.001861
263337671,0.002179,0.013056,0.929887,0.245605,0.003417
263337671,0.003771,0.027332,0.897167,0.277175,0.007390
263337671,0.005565,0.028687,0.923941,0.144937,0.002416
263337671,0.002657,0.010396,0.937068,0.210296,0.001473
263337671,0.002572,0.023736,0.930956,0.195460,0.001471
263337671,0.024971,0.016409,0.590290,0.566074,0.002671
263337671,0.002346,0.017249,0.905677,0.290867,0.003571


In [37]:
def final_pred(row):
#     if (row['Distinct'] > 1
#         and (
#             not isinstance(row['Decision'], str)
#             and (
#                 (row['av'] in ('E', 'S'))
#                 or (row['md'] in ('E', 'S'))
#                 or (row['ch'] in ('E', 'S'))
#                 or (row['as'] in ('E', 'S'))
#                 or (row['mk'] in ('E', 'S'))
#                 or (row['et'] in ('E', 'S'))
#             )
#         )
#        ):
#         return '?'
    
    if (row['disp_E'] > 0):
        return 'E'
    else:
        maxpred = 'disp_E'
        for c in ['disp_N', 'disp_J', 'disp_S', 'disp_B']:
            if row[c] > row[maxpred]:
                maxpred = c
        return maxpred[5]

agg_preds = pd.read_csv('~/Astronet-Triage/Labels - extended mission test.csv', header=0, low_memory=False)
agg_preds = agg_preds.set_index('TIC ID')
agg_preds = final_preds.join(agg_preds)
agg_preds['final'] = agg_preds.apply(final_pred, axis=1)
agg_preds = agg_preds[['final']]

In [38]:
agg_preds[agg_preds['final'] == '?']

,final
tic_id,


In [39]:
agg_preds.to_csv('~/Astronet-Triage/tces-s33_cam1_sample-preds.csv')